In [ ]:


import numpy as np 
import pandas as pd 



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


ModuleNotFoundError: No module named 'pandas'

In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.0 MB 9.9 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/11.0 MB 7.3 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/11.0 MB 5.5 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.0 MB 5.5 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/11.0 MB 5.4 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.0 MB 5.2 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.0 MB 4.8 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.0 MB 4.6 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.0 MB 4.2 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.0 MB 3.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 3.8 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 3.6 MB/s eta 0:00:01
   ---

In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
import re

In [80]:
booking_df = pd.read_csv('D:/booking_hotel.csv', encoding='latin1')
tripadvisor_df = pd.read_csv('D:/tripadvisor_room.csv', encoding='latin1')


In [81]:
print("Booking Data:\n", booking_df.head())
print("TripAdvisor Data:\n", tripadvisor_df.head())

Booking Data:
                           Hotel Name                       Location Rating  \
0   Krabi La Playa Resort - SHA Plus                  Ao Nang Beach    8.2   
1        KC Beach Club & Pool Villas  Chaweng City Center , Chaweng    7.1   
2       Rawai VIP Villas & Kids Park                    Rawai Beach    8.3   
3    FuramaXclusive Sathorn, Bangkok              Bang Rak, Bangkok    7.7   
4  Bo Phut Resort and Spa - SHA Plus                         Bophut    9.3   

  Review Score review_count  Room Score  \
0    Very Good          141         8.6   
1         Good          193         NaN   
2    Very Good           63         8.9   
3         Good        1,304         8.1   
4    Wonderful          294         9.6   

                               Room Type                Bed Type  Room Price  
0             Deluxe Double or Twin Room     1 double or 2 twins     146,026  
1  Double Room with Balcony and Sea View             1 queen bed     215,304  
2                   

In [ ]:

def clean_price(price):
    if isinstance(price, str):
        try:
         
            return float(price.replace(',', '').replace('BDT', '').strip())
        except ValueError:
         
            return np.nan
    return price


booking_df['Room Price'] = booking_df['Room Price'].apply(clean_price)
tripadvisor_df['Room Price'] = tripadvisor_df['Room Price'].apply(clean_price)

print("Booking DataFrame after cleaning:\n", booking_df.head())
print("\nTripadvisor DataFrame after cleaning:\n", tripadvisor_df.head())

# # Hàm để xử lý số lượng đánh giá
# def clean_review_count(value):
#     if isinstance(value, str):
#         # Thay dấu phẩy thành dấu chấm cho các số thập phân
#         value = value.replace(',', '.')
#         # Trích xuất phần số bằng cách loại bỏ chữ
#         number = ''.join([char for char in value if char.isdigit() or char == '.'])
#         try:
#             # Chuyển thành số thực
#             return float(number)
#         except ValueError:
#             # Trả về NaN nếu không thể chuyển đổi
#             return float('nan')
#     else:
#         # Nếu giá trị không phải là chuỗi, trả về NaN
#         return float('nan')

# # Áp dụng hàm vào cột review_count cho booking_df và tripadvisor_df
# booking_df['review_count'] = booking_df['review_count'].apply(clean_review_count)
# tripadvisor_df['review_count'] = tripadvisor_df['review_count'].apply(clean_review_count)


# # Xử lý điểm đánh giá
# booking_df['Review Score'] = pd.to_numeric(booking_df['Review Score'], errors='coerce')

# # Loại bỏ các hàng có giá trị NaN
# booking_df.dropna(subset=['Room Price', 'Review Score', 'review_count'], inplace=True)
# tripadvisor_df.dropna(subset=['Room Price', 'review_count'], inplace=True)

# # Kiểm tra kết quả sau khi làm sạch
# print("Booking DataFrame after cleaning:\n", booking_df.head())
# print("\nTripadvisor DataFrame after cleaning:\n", tripadvisor_df.head())

Booking DataFrame after cleaning:
                           Hotel Name                       Location Rating  \
0   Krabi La Playa Resort - SHA Plus                  Ao Nang Beach    8.2   
1        KC Beach Club & Pool Villas  Chaweng City Center , Chaweng    7.1   
2       Rawai VIP Villas & Kids Park                    Rawai Beach    8.3   
3    FuramaXclusive Sathorn, Bangkok              Bang Rak, Bangkok    7.7   
4  Bo Phut Resort and Spa - SHA Plus                         Bophut    9.3   

  Review Score review_count  Room Score  \
0    Very Good          141         8.6   
1         Good          193         NaN   
2    Very Good           63         8.9   
3         Good        1,304         8.1   
4    Wonderful          294         9.6   

                               Room Type                Bed Type  Room Price  
0             Deluxe Double or Twin Room     1 double or 2 twins    146026.0  
1  Double Room with Balcony and Sea View             1 queen bed    215304.0  


In [ ]:
# Hàm để xử lý số lượng đánh giá
def clean_review_count(value):
    if isinstance(value, str):
        try:
            # Loại bỏ ký tự không cần thiết và chuyển đổi sang float
            return float(value.replace(',', '').replace('viewer', '').strip())
        
        except ValueError:
            # Nếu không thể chuyển đổi, trả về NaN
            return np.nan
    return value

# Áp dụng hàm vào cột review_count cho booking_df và tripadvisor_df
booking_df['review_count'] = booking_df['review_count'].apply(clean_review_count)
tripadvisor_df['review_count'] = tripadvisor_df['review_count'].apply(clean_review_count)

# # Xử lý điểm đánh giá
# booking_df['Review Score'] = pd.to_numeric(booking_df['Review Score'], errors='coerce')

# Loại bỏ các hàng có giá trị NaN
booking_df.dropna(subset=['Room Price', 'Review Score', 'review_count'], inplace=True)
tripadvisor_df.dropna(subset=['Room Price', 'review_count'], inplace=True)

# Kiểm tra kết quả sau khi làm sạch
print("Booking DataFrame after cleaning:\n", booking_df.head())
print("\nTripadvisor DataFrame after cleaning:\n", tripadvisor_df.head())

Booking DataFrame after cleaning:
 Empty DataFrame
Columns: [Hotel Name, Location, Rating, Review Score, review_count, Room Score, Room Type, Bed Type, Room Price]
Index: []

Tripadvisor DataFrame after cleaning:
                               property name  Room Price  review_count  \
0       1. SALA Samui Choengmon Beach Resort     45,854        2820.0   
1     2. Napasai, A Belmond Hotel, Koh Samui     29,666        1191.0   
2                  3. Pimalai Resort and Spa     31,667        3287.0   
3  4. Avani+ Mai Khao Phuket Suites & Villas     23,887         144.0   
4                   5. Sheraton Samui Resort     18,816        1006.0   

                                  Comment about room  
0  The breakfast is excellent, the pools are real...  
1                We a had perfect view at the beach,  
2                                  It has everything  
3                                 the villa was very  
4  Me & my wife stayed there for 3 Nights and we ...  


In [33]:
avg_price = tripadvisor_df['Room Price'].mean()
avg_reviews = tripadvisor_df['review_count'].mean()

tripadvisor_df['Preferred'] = ((tripadvisor_df['Room Price'] > avg_price) &
                               (tripadvisor_df['review_count'] > avg_reviews)).astype(int)


TypeError: Could not convert string '2,8201,1913,2871441,0069681,4742571,6068844,8783,0503,7361,2101,4489,7711,1123,0427802,2212233436301,4762531846581,5733,2302,8721,7333,4172077721,6271,8432,9646959,8353,3791,5491541931,0582,9761,1292031,7052,5271,9236221636693266431,4535851983199084661,7626022,6034584,4562,8222,5762589217852,9041,2065121,2511,2469402,4221,2581,5321,3101,5453838801,3382222973673495,5261,4381,5742702,4824,0583,4631,3627,0844,9751,9019176404,9895122,9341,6658397441911021,1481,1665101,9121,0381,6651545718052,6053431941,1801,1541,1424,9043,0675425,8473,2292,6463,3721,5483913,8546363,1056332,9258058083774222357343312,0761,759822102654,5161,8924,9283,95011,6522,3984221,8923,2751,1772,4398092,1653,871185617431,12935847234117772880559203535851,1235,6952,7293761,2505041,3241403929072812,4284063,0952434,6854,4666978661,0912,8441,8513,4782,3401,4291,0162,2502,0501,5211,7812192,4337861,1283,1755,1093,2884042,2561,7434,2443,5301,7815598,0029983,9573,0803,8821,6031201,0662,0932273

In [19]:
X_text = tripadvisor_df['Comment about room']
X_numeric = tripadvisor_df[['Room Price', 'review_count']]
y = tripadvisor_df['Preferred']

In [20]:
tfidf = TfidfVectorizer(max_features=500)
X_text_tfidf = tfidf.fit_transform(X_text).toarray()

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
X = np.hstack((X_text_tfidf, X_numeric))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))